!pip install pychrome selenium kaggle
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt update -y
!apt install ./google-chrome-stable_current_amd64.deb -y
import os
version=os.popen('google-chrome --version').read().replace('Google Chrome ','').replace(' \n','')
!pip install chromedriver-py=={version}
!rm ./google-chrome-stable_current_amd64.deb

In [2]:
!pip install pychrome

  Created wheel for pychrome: filename=pychrome-0.2.3-py2.py3-none-any.whl size=8143 sha256=4fb5a116a9aa8428c47742d6044acbd0eec28041561e2a54f31cb1724c9d3dfa
  Stored in directory: c:\users\shiva\appdata\local\pip\cache\wheels\fa\8b\df\f6d72f867a6d357e59abdaabeb15bd365d45ed515762db589d
Successfully built pychrome


In [3]:
import os
version=os.popen('google-chrome --version').read().replace('Google Chrome ','').replace(' \n','')
version

''

In [1]:
from chromedriver_py import binary_path
import pychrome
from selenium import webdriver
from selenium.webdriver.common.service import Service
from selenium.webdriver.common.by import By
import json
import pandas as pd
from time import sleep

ModuleNotFoundError: No module named 'chromedriver_py'

In [3]:
url='https://tradingeconomics.com/commodity/crude-oil'

In [4]:
# Create a new instance of the Chrome driver
options = webdriver.ChromeOptions()
options.add_argument('--headless')        #remove this for easy debbuing on your laptop /pc
options.add_argument('--no-sandbox')                             
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--remote-debugging-port=8000')

driver = webdriver.Chrome(binary_path,options=options)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [5]:
dev_tools = pychrome.Browser(url='http://localhost:8000')
tab=dev_tools.list_tab()[0]
tab.start()
tab.Network.enable()

{}

In [6]:
def response_received(**kwargs):
    if kwargs.get('response').get('url').find('https://markets.tradingeconomics.com/chart') >= 0:
        global raw_data
        raw_data = tab.Network.getResponseBody(requestId=kwargs['requestId'])
        # print(tab.Network.getResponseBody(requestId=kwargs['requestId']))
tab.Network.responseReceived = response_received
driver.get(url)
sleep(5)
# driver.find_element(by=By.XPATH, value="//a[@title='All']").click()
driver.find_element_by_xpath("//*[@title='All']").click()
# driver.find_element_by_xpath("//*[@data-span='max']").click()
sleep(5)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  # Remove the CWD from sys.path while we load stuff.


In [7]:
data = json.loads(raw_data['body'])

In [8]:
df=pd.DataFrame(data['series'][0]['data'])
df.head()

,date,x,y,percentChange,change
0,1983-03-30T00:00:00,417830400000,29.27,NaN,NaN
1,1983-04-04T00:00:00,418262400000,30.63,4.646396,1.36
2,1983-05-02T00:00:00,420681600000,30.25,-1.240614,-0.38
3,1983-06-01T00:00:00,423273600000,31.38,3.735537,1.13
4,1983-07-01T00:00:00,425865600000,32.00,1.975781,0.62


In [9]:
df.rename(columns={'y':'price'},inplace=True)
df.drop(['x'],axis=1,inplace=True)

In [10]:
df.to_csv(f'crude-oil-price.csv',index=None)